In [64]:
import numpy as np
import os
import pickle
import torch
import gpytorch

In [65]:
# Load ALL the the results
jar_of_pickles = os.listdir("results/parameter_stuff")
jar_of_pickles.sort()
jar_of_pickles = [p for p in jar_of_pickles if p.endswith(".pickle")]
all_results = []
for pickl in jar_of_pickles:
    with open(pickl, "rb") as f:
        test = pickle.load(f)
        all_results.append(test)

In [66]:
param_lists = {"SE": {"raw_lengthscale" : []},
            "MAT52": {"raw_lengthscale" : []},
            "MAT32": {"raw_lengthscale" : []},
            "RQ": {"raw_lengthscale" : [], 
                    "raw_alpha" : []},
            "PER":{"raw_lengthscale": [], "raw_period_length":[]},
            "LIN":{"raw_variance" : []},
            "C":{"raw_outputscale":[]},
            "noise": {"raw_noise":[]}}

In [67]:
def extract_param_values(kernel_name, params_result, constraint = None):
    temp = {"SE": {"raw_lengthscale" : []},
            "MAT52": {"raw_lengthscale" : []},
            "MAT32": {"raw_lengthscale" : []},
            "RQ": {"raw_lengthscale" : [], 
                    "raw_alpha" : []},
            "PER":{"raw_lengthscale": [], "raw_period_length":[]},
            "LIN":{"raw_variance" : []},
            "C":{"raw_outputscale":[]},
            "noise": {"raw_noise":[]}}

    kernel_name = kernel_name.split("_")[-1].split(".")[0]
    
    if constraint is None:
        constraint = lambda x:x
    elif constraint == "positive":
        positive = gpytorch.constraints.Positive()
        constraint = positive.transform
    else:
        constraint = constraint
    
    temp["noise"]["raw_noise"].append(constraint(params_result[0][1]).item())
    
    # Here I have a "0" and "1" in the kernels, in particular 2 scale kernels
    if "+" in kernel_name:
        # Kernel 0
        n1 = kernel_name.split("+")[0]
        n1_params = [p for p in params_result if "0" in p[0]]
        for p in n1_params:
            if "outputscale" in p[0].split(".")[-1]:
                temp["C"][p[0].split(".")[-1]].append(constraint(p[1]).item())
                continue
            temp[n1][p[0].split(".")[-1]].append(constraint(p[1]).item())
        
        n2 = kernel_name.split("+")[1]
        n2_params = [p for p in params_result if "1" in p[0]]
        for p in n2_params:
            if "outputscale" in p[0].split(".")[-1]:
                temp["C"][p[0].split(".")[-1]].append(constraint(p[1]).item())
                continue
            temp[n2][p[0].split(".")[-1]].append(constraint(p[1]).item())

        
    elif "*" in kernel_name:
        param_lists["C"]["raw_outputscale"].append(constraint(params_result[1][1]).item())
        
        n1 = kernel_name.split("*")[0]
        n1_params = [p for p in params_result if "0" in p[0]]
        for p in n1_params:
            temp[n1][p[0].split(".")[-1]].append(constraint(p[1]).item())
        
        n2 = kernel_name.split("*")[1]
        n2_params = [p for p in params_result if "1" in p[0]]
        for p in n2_params:
            temp[n2][p[0].split(".")[-1]].append(constraint(p[1]).item())

        
        
    
    

    return temp
    
    

In [68]:
def extend_current_dict(old, new_data):
    # We assume the exact same structure of both
    # In both we have dicts of dicts and at some point lists which we append to
    for key in old:
        for inner_key in old[key]:
            old[key][inner_key] = old[key][inner_key] + new_data[key][inner_key]
    return old

In [69]:
all_param_values = {"SE": {"raw_lengthscale" : []},
            "MAT52": {"raw_lengthscale" : []},
            "MAT32": {"raw_lengthscale" : []},
            "RQ": {"raw_lengthscale" : [], 
                    "raw_alpha" : []},
            "PER":{"raw_lengthscale": [], "raw_period_length":[]},
            "LIN":{"raw_variance" : []},
            "C":{"raw_outputscale":[]},
            "noise": {"raw_noise":[]}}
for name, result in zip(jar_of_pickles, all_results):
    for train_run in result:
        #extend_current_dict(all_param_values, extract_param_values(name, result[train_run], constraint="positive"))
        extend_current_dict(all_param_values, extract_param_values(name, result[train_run], constraint=None))


In [70]:
# Calculate mean + std of all raw values
#print(f"Lengthscale Mean: {np.mean(all_param_values["SE"]["raw_lengthscale"])}, Std: {np.std(all_param_values["SE"]["raw_lengthscale"])}")
print(f"SE-length: \t{np.mean(all_param_values['SE']['raw_lengthscale'])}, \t{np.std(all_param_values['SE']['raw_lengthscale'])}")
print(f"MAT32-length: \t{np.mean(all_param_values['MAT32']['raw_lengthscale'])}, \t{np.std(all_param_values['MAT32']['raw_lengthscale'])}")
print(f"MAT52-length: \t{np.mean(all_param_values['MAT52']['raw_lengthscale'])}, \t{np.std(all_param_values['MAT52']['raw_lengthscale'])}")
print(f"RQ-length: \t{np.mean(all_param_values['RQ']['raw_lengthscale'])}, \t{np.std(all_param_values['RQ']['raw_lengthscale'])}")
print(f"RQ-length: \t{np.mean(all_param_values['RQ']['raw_alpha'])}, \t{np.std(all_param_values['RQ']['raw_alpha'])}")
print(f"PER-length: \t{np.mean(all_param_values['PER']['raw_lengthscale'])}, \t{np.std(all_param_values['PER']['raw_lengthscale'])}")
print(f"PER-period: \t{np.mean(all_param_values['PER']['raw_period_length'])}, \t{np.std(all_param_values['PER']['raw_period_length'])}")
print(f"LIN-variance: \t{np.mean(all_param_values['LIN']['raw_variance'])}, \t{np.std(all_param_values['LIN']['raw_variance'])}")
print(f"C-scale: \t{np.mean(all_param_values['C']['raw_outputscale'])}, \t{np.std(all_param_values['C']['raw_outputscale'])}")
print(f"Noise-noise: \t{np.mean(all_param_values['noise']['raw_noise'])}, \t{np.std(all_param_values['noise']['raw_noise'])}")





#all_param_values["SE"]["raw_lengthscale"]

SE-length: 	-0.21221139138922668, 	1.8895426067756804
MAT32-length: 	1.5711054238673443, 	2.4453761235991216
MAT52-length: 	0.7993038925994188, 	2.145122566357853
RQ-length: 	-0.049841950913676276, 	1.9426354614713097
RQ-length: 	1.882148553921053, 	3.096431944989054
PER-length: 	0.7778461197268618, 	2.288946656544974
PER-period: 	0.6485334993738499, 	0.9930632050553377
LIN-variance: 	-0.8017903983055685, 	0.9966569921354465
C-scale: 	-1.6253091096349706, 	2.2570021716661923
Noise-noise: 	-3.51640656386717, 	3.5831320474767407


In [71]:
print(f"""prior_dict = {{'SE': {{'raw_lengthscale' : {{"mean": {np.mean(all_param_values['SE']['raw_lengthscale'])} , "std":{np.std(all_param_values['SE']['raw_lengthscale'])}}}}},
            'MAT52': {{'raw_lengthscale' :{{"mean": {np.mean(all_param_values['MAT52']['raw_lengthscale'])}, "std":{np.std(all_param_values['MAT52']['raw_lengthscale'])} }} }},
            'MAT32': {{'raw_lengthscale' :{{"mean": {np.mean(all_param_values['MAT32']['raw_lengthscale'])}, "std":{np.std(all_param_values['MAT32']['raw_lengthscale'])} }} }},
            'RQ': {{'raw_lengthscale' :{{"mean": {np.mean(all_param_values['RQ']['raw_lengthscale'])}, "std":{np.std(all_param_values['RQ']['raw_lengthscale'])} }}, 
                    'raw_alpha' :{{"mean": {np.mean(all_param_values['RQ']['raw_alpha'])}, "std":{np.std(all_param_values['RQ']['raw_alpha'])} }} }},
            'PER':{{'raw_lengthscale':{{"mean": {np.mean(all_param_values['PER']['raw_lengthscale'])}, "std":{np.std(all_param_values['PER']['raw_lengthscale'])} }},
                    'raw_period_length':{{"mean": {np.mean(all_param_values['PER']['raw_period_length'])}, "std":{np.std(all_param_values['PER']['raw_period_length'])} }} }},
            'LIN':{{'raw_variance' :{{"mean": {np.mean(all_param_values['LIN']['raw_variance'])}, "std":{np.std(all_param_values['LIN']['raw_variance'])} }} }},
            'C':{{'raw_outputscale':{{"mean": {np.mean(all_param_values['C']['raw_outputscale'])}, "std":{np.std(all_param_values['C']['raw_outputscale'])} }} }},
            'noise': {{'raw_noise':{{"mean": {np.mean(all_param_values['noise']['raw_noise'])}, "std":{np.std(all_param_values['noise']['raw_noise'])} }} }} }}""")

prior_dict = {'SE': {'raw_lengthscale' : {"mean": -0.21221139138922668 , "std":1.8895426067756804}},
            'MAT52': {'raw_lengthscale' :{"mean": 0.7993038925994188, "std":2.145122566357853 } },
            'MAT32': {'raw_lengthscale' :{"mean": 1.5711054238673443, "std":2.4453761235991216 } },
            'RQ': {'raw_lengthscale' :{"mean": -0.049841950913676276, "std":1.9426354614713097 }, 
                    'raw_alpha' :{"mean": 1.882148553921053, "std":3.096431944989054 } },
            'PER':{'raw_lengthscale':{"mean": 0.7778461197268618, "std":2.288946656544974 },
                    'raw_period_length':{"mean": 0.6485334993738499, "std":0.9930632050553377 } },
            'LIN':{'raw_variance' :{"mean": -0.8017903983055685, "std":0.9966569921354465 } },
            'C':{'raw_outputscale':{"mean": -1.6253091096349706, "std":2.2570021716661923 } },
            'noise': {'raw_noise':{"mean": -3.51640656386717, "std":3.5831320474767407 } } }
